* CRIM - уровень преступности на душу населения
* ZN - доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.
* INDUS - доля акров неторгового бизнеса на город.
* CHAS - 1, если граничит с рекой, 0 иначе
* NOX - концентрация оксидов азота (частей на 10 миллионов)
* RM - среднее количество комнат в квартире
* AGE - доля жилых домов, построенных до 1940 г.
* DIS - взвешенные расстояния до пяти центров занятости Бостона
* RAD - индекс доступности к радиальным магистралям
* TAX - полная ставка налога на имущество за 10 000 долларов США
* PTRATIO - соотношение учеников и учителей по городам
* B - 1000(Bk - 0.63)^2, где Bk доля чернокожих людей по городам
* LSTAT - % более низкого статуса населения
* MEDV - Средняя стоимость домов в 1000 долларов.

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [1]:
import sqlite3
import pandas as pd

con = sqlite3.connect(":memory:")
cur = con.cursor()

link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

data.to_sql("boston", con, index=False, if_exists="replace")

pd.read_sql(
    """
    SELECT *
    FROM boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


## 1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [2]:
# Будем считать что нули это тоже пустые значения, т.к. выглядит подозрительно :) 

pd.read_sql(
    """
    SELECT 'CRIM',COUNT(CRIM) FROM boston Where (CRIM is null or CRIM = '' or CRIM = 0.0)
    UNION
    SELECT 'ZN',COUNT(ZN) FROM boston Where (ZN is null or ZN = '' or ZN = 0.0)
    UNION
    SELECT 'INDUS',COUNT(INDUS) FROM boston Where (INDUS is null or INDUS = '' or INDUS = 0.0)
    UNION
    SELECT 'CHAS',COUNT(CHAS) FROM boston Where (CHAS is null or CHAS = '' or CHAS = 0.0)
    UNION
    SELECT 'NOX',COUNT(NOX) FROM boston Where (NOX is null or NOX = '' or NOX = 0.0)
    """,
    con,
)

,'CRIM',COUNT(CRIM)
0,CHAS,471
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,372


## 2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [3]:
pd.read_sql(
    """
    SELECT 'CRIM',COUNT(DISTINCT CRIM) FROM boston
    UNION
    SELECT 'ZN',COUNT(DISTINCT ZN) FROM boston
    UNION
    SELECT 'INDUS',COUNT(DISTINCT INDUS) FROM boston
    UNION
    SELECT 'CHAS',COUNT(DISTINCT CHAS) FROM boston
    UNION
    SELECT 'NOX',COUNT(DISTINCT NOX) FROM boston
    """,
    con,
)

,'CRIM',COUNT(DISTINCT CRIM)
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


## 3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [4]:
sql_temp = '''
    DROP TABLE IF EXISTS temp;
    
    CREATE TABLE temp AS 
    SELECT 
        'CRIM',
        AVG(CRIM),
        MIN(CRIM),
        CASE 
            WHEN AVG(CRIM) = MIN(CRIM) THEN 'Да' 
            ELSE 'Нет' 
        END AS Condition FROM boston
    UNION
    SELECT 
        'ZN',
        AVG(ZN),
        MIN(ZN),
        CASE 
            WHEN AVG(ZN) = MIN(ZN) THEN 'Да' 
            ELSE 'Нет' 
        END AS Condition FROM boston
    UNION
    SELECT 
        'INDUS',
        AVG(INDUS),
        MIN(INDUS),
        CASE 
            WHEN AVG(INDUS) = MIN(INDUS) THEN 'Да' 
            ELSE 'Нет' 
        END AS Condition FROM boston
    UNION
    SELECT 
        'CHAS',
        AVG(CHAS),
        MIN(CHAS),
        CASE 
            WHEN AVG(CHAS) = MIN(CHAS) THEN 'Да' 
            ELSE 'Нет' 
        END AS Condition FROM boston
    UNION
    SELECT 
        'NOX',
        AVG(NOX),
        MIN(NOX),
        CASE 
            WHEN AVG(NOX) = MIN(NOX) THEN 'Да' 
            ELSE 'Нет' 
        END AS Condition FROM boston;
    '''

In [5]:
cur.executescript(sql_temp)
con.commit()

pd.read_sql(
    """
    SELECT * FROM temp
    WHERE Condition = 'Да'
    """
    ,con,
)

,'CRIM',AVG(CRIM),MIN(CRIM),Condition


* для указанного набора жилья в данных районах :
* CRIM - уровень преступности на душу населения - не является минимальным для совокупности районов (остерегаться преступности, сравнить уровень преступности для иных районов)
* ZN - доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов - присутствуют благоприятные большие зоны для застройки под жилье (оценить наиболее большие зоны для покупки)
* INDUS - доля акров неторгового бизнеса на город - присутствуют благоприятные большие зоны для застройки под заводы,станции либо стоит опасаться районов с слишком большой концентрацией заводов для покупки жилья (оценить зоны для покупки)(провести аудит, оценить обстановку для закупку) 
* CHAS - 1, если граничит с рекой, 0 иначе - нельзя сказать что абсолютно все районы расположены только вдоль реки или что все районы не имеют границ с рекой (оценить  дома с границей с рекой для покупки)
* NOX - концентрация оксидов азота (частей на 10 миллионов) - не является минимальной в данных районах (требуется анализ наиболее не благоприятных районах с превышением концентрации) 

## 4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [6]:
pd.read_sql(
    """
    WITH 
    min_25  AS (SELECT RM FROM boston ORDER BY MEDV      limit 25),
    max_25  AS (SELECT RM FROM boston ORDER BY MEDV DESC limit 25),
    min_50  AS (SELECT RM FROM boston ORDER BY MEDV      limit 50),
    max_50  AS (SELECT RM FROM boston ORDER BY MEDV DESC limit 50),
    min_100 AS (SELECT RM FROM boston ORDER BY MEDV      limit 100),
    max_100 AS (SELECT RM FROM boston ORDER BY MEDV DESC limit 100),
    min_200 AS (SELECT RM FROM boston ORDER BY MEDV      limit 200),
    max_200 AS (SELECT RM FROM boston ORDER BY MEDV DESC limit 200),
    min_300 AS (SELECT RM FROM boston ORDER BY MEDV      limit 300),
    max_300 AS (SELECT RM FROM boston ORDER BY MEDV DESC limit 300)
    
    SELECT 
        '25' AS Houses,  
        AVG(min25.RM) AS AvgRooms,
        AVG(max25.RM) AS AvgPrice,
        AVG(min25.RM)-AVG(max25.RM) AS Diff
    FROM min_25 as min25,max_25 as max25
    UNION ALL
    SELECT 
        '50',
        AVG(min50.RM),
        AVG(max50.RM),
        AVG(min50.RM)-AVG(max50.RM) AS diff
    FROM min_50 as min50,max_50 as max50
    UNION ALL
    SELECT 
        '100',
        AVG(min100.RM),
        AVG(max100.RM),
        AVG(min100.RM)-AVG(max100.RM) AS diff
    FROM min_100 as min100,max_100 as max100
    UNION ALL
    SELECT 
        '200',
        AVG(min200.RM),
        AVG(max200.RM),
        AVG(min200.RM)-AVG(max200.RM) AS diff
    FROM min_200 as min200,max_200 as max200
    UNION ALL
    SELECT 
        '300',
        AVG(min300.RM),
        AVG(max300.RM),
        AVG(min300.RM)-AVG(max300.RM) AS diff
    FROM min_300 as min300,max_300 as max300
    """,
    con,
)

,Houses,AvgRooms,AvgPrice,Diff
0,25,5.747840,7.637320,-1.889480
1,50,5.753240,7.487060,-1.733820
2,100,5.887120,7.198760,-1.311640
3,200,5.911705,6.758680,-0.846975
4,300,5.972227,6.539007,-0.566780


* В результате -
* при увеличении количества рассматриваемых домов 
* среднее количество комнат увеличивается, а средняя цена начинает падает.
* в среднем, чем больше комнат - тем меньше цена за комнату(дом).

## 5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [7]:
pd.read_sql(
    """
    WITH maxMEDV AS (SELECT MAX(MEDV) AS MAX FROM boston)
    
    SELECT 
        boston.LSTAT,
        boston.MEDV,
        RANK() OVER(PARTITION BY MEDV ORDER BY LSTAT) AS 'Rank' 
    FROM boston,maxMEDV
    WHERE boston.MEDV = maxMEDV.MAX
    """,
    con,
)

,LSTAT,MEDV,Rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,3
3,2.96,50.0,4
4,2.97,50.0,5
5,3.16,50.0,6
6,3.26,50.0,7
7,3.32,50.0,8
8,3.70,50.0,9
9,3.73,50.0,10


* Найдено 16 объектов с максимальной стоимостью, каждый из который имеет ранг,
* определяющий благоприятность покупки при ориентировании на процент населения с более низким статусом.
* Приоритет покупки стоит отдавать объектам с наилучшим рангом (от 1)

## 6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [8]:
pd.read_sql(
    """
    SELECT 
        'Граничит' AS State,
        AVG(MEDV) AS AvgPrice 
    FROM boston 
    WHERE CHAS = 1
    UNION
    SELECT 
        'Не граничит',
        AVG(MEDV) 
    FROM boston 
    WHERE CHAS = 0
    """,
    con,
)

,State,AvgPrice
0,Граничит,28.440000
1,Не граничит,22.093843


In [9]:
pd.read_sql(
    """
    SELECT DISTINCT AVG(MEDV) OVER(PARTITION BY CHAS ORDER BY CHAS)
    FROM boston 
    """,
    con,
)

,AVG(MEDV) OVER(PARTITION BY CHAS ORDER BY CHAS)
0,22.093843
1,28.440000


* Цена увеличивается если есть граница с рекой

## 7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [10]:
pd.read_sql(
    """
    WITH resultTable AS
    (
    SELECT 
        'CRIM',
        (SELECT AVG(CRIM) FROM boston WHERE CHAS=1)> AVG(CRIM) AS result
    FROM boston
    UNION ALL
    SELECT 
        'ZN',
        (SELECT AVG(ZN) FROM boston WHERE CHAS=1) > AVG(ZN) AS result 
    FROM boston
    UNION ALL
    SELECT 
        'INDUS',
        (SELECT AVG(INDUS) FROM boston WHERE CHAS=1) > AVG(INDUS) AS result 
    FROM boston
    UNION ALL
    SELECT 
        'CHAS',
        (SELECT AVG(CHAS) FROM boston WHERE CHAS=1) > AVG(CHAS) AS result 
    FROM boston
    UNION ALL
    SELECT 
        'NOX',
        (SELECT AVG(NOX) FROM boston WHERE CHAS=1) > AVG(NOX) AS result 
    FROM boston
    )
    
    SELECT * FROM resultTable
    WHERE result = 1
    """,
    con,
)

,'CRIM',result
0,INDUS,1
1,CHAS,1
2,NOX,1


* В местах, граничащие с рекой, увеличивается как доля акров неторгового бизнеса на город, так и концентрация оксидов азота (частей на 10 миллионов).
* Можно предположить что местные предприятия сливают отходы в реку или используют иные технологии, которые повышают концентрацию оксидов азота.

## 8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [11]:
pd.read_sql(
    """
    WITH IndusPerc AS
    (SELECT 
        INDUS, 
        NTILE(10) OVER (ORDER BY INDUS) AS Percentile
    FROM boston),
    NoxPerc AS
    (SELECT 
        NOX, 
        NTILE(10) OVER (ORDER BY NOX) AS Percentile
    FROM boston)
          
   SELECT 
       NoxPerc.Percentile,
       IndusPerc.INDUS,
       NoxPerc.NOX 
   FROM IndusPerc
   LEFT JOIN NoxPerc
   ON IndusPerc.Percentile = NoxPerc.Percentile
   GROUP BY IndusPerc.Percentile
    """,
    con,
)

,Percentile,INDUS,NOX
0,1,0.46,0.3850
1,2,2.93,0.4280
2,3,4.39,0.4429
3,4,5.96,0.4720
4,5,7.38,0.5070
5,6,9.69,0.5380
6,7,13.89,0.5800
7,8,18.10,0.6090
8,9,18.10,0.6710
9,10,19.58,0.7130


* Прослеживается прямая зависимость между увеличением застройкой неторгового бизнесе(заводы, станции и т.д.) и увеличением концентрации оксида азота.